#HEART ATTACK ANALYSIS PREDICTION

###Kelompok 13 Kecerdasan Buatan :

Faizah Bestiyana Darmawati (20/456364/TK/50494)         
Putri Ayu Shintaningrum (20/460559/TK/51148)    
Resiyanda Wafa Ramadaningtyas Aribowo (20/460563/TK/51152)   
Tasya Nafisah Kamal (20/460569/TK/51158)




In [22]:
# Mengimport library yang dibutuhkan
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import pandas as pd
import numpy as np
import csv

In [23]:
# Memvisualisasikan dataset yang digunakan
train_df = pd.read_csv('heart.csv')
train_df.head(5)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [24]:
# Mendefinisikan dataset yang digunakan
def load_csv(filename):
	dataset = list()
	with open(filename, 'heart.csv') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [25]:
# Mengubah kolom string ke integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [26]:
# Membagi dataset ke k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [27]:
# Menghitung presentase akurasi
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [28]:
# Mengevaluasi algoritma dengan menggunakan cross validation split 
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [29]:
# Membagi dataset berdasarkan atribute dan nilai atribute 
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [30]:
# Menghitung Gini index untuk membagi dataset
def gini_index(groups, classes):
	# Menghitung semua sample di titik pembagian
	n_instances = float(sum([len(group) for group in groups]))
	# Menjumlahkan Gini index untuk setiap grup
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# Menghindari terjadinya pembagian dengan nilai 0
		if size == 0:
			continue
		score = 0.0
		# Memberi skor kepada grup berdasarkan skor di tiap kelas 
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# Menghitung skor tiap grup berdasarkan relatif sizenya
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [31]:
# Memilih titik pembagi terbaik untuk dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [32]:
# Membuat nilai node terminal
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [33]:
# Membuat child splits untuk node atau membuat terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# Cek jika tidak ada splits
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# Cek untuk kedalaman maksimal
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# Proses child sebelah kiri
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# Proses child sebelah kanan
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

In [34]:
# Membentuk decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

In [35]:
# Membuat prediksi dengan decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [36]:
# Membuat subsample acak dari dataset dengan pengubahan
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

In [37]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

In [38]:
# Algoritma random forest
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

In [39]:
# Tes algoritma random forest
seed(2)
# Memuat dan mempersiapkan data
filename = r'heart.csv'
dataset = csv.reader(open(filename, "rt"))
dataset = list(dataset)
# Mengubah kelas kolom ke integer 
str_column_to_int(dataset, len(dataset[0])-1)
# Menjalankan algoritma
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 3, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [73.33333333333333, 83.33333333333334, 80.0, 70.0, 70.0]
Mean Accuracy: 75.333%
Trees: 3
Scores: [73.33333333333333, 78.33333333333333, 76.66666666666667, 83.33333333333334, 70.0]
Mean Accuracy: 76.333%
Trees: 5
Scores: [80.0, 80.0, 78.33333333333333, 70.0, 85.0]
Mean Accuracy: 78.667%
Trees: 10
Scores: [81.66666666666667, 80.0, 83.33333333333334, 83.33333333333334, 75.0]
Mean Accuracy: 80.667%


In [40]:
# Confusion matrix
def confusion_matrix(predicted,actual): #pass predicted and original labels to this function
    matrix=np.zeros((2,2)) # form an empty matric of 2x2
    for i in range(len(predicted)): #the confusion matrix is for 2 classes: 1,0
        #1=positive, 0=negative
        if int(predicted[i])==1 and int(actual[i])==0: 
            matrix[0,0]+=1 #True Positives
        elif int(predicted[i])==-1 and int(actual[i])==1:
            matrix[0,1]+=1 #False Positives
        elif int(predicted[i])==0 and int(actual[i])==1:
            matrix[1,0]+=1 #False Negatives
        elif int(predicted[i])==0 and int(actual[i])==0:
            matrix[1,1]+=1 #True Negatives
    precision=matrix[0,0]/(matrix[0,0]+matrix[0,1])
    print("Precision:",precision)
    recall=matrix[0,0]/(matrix[0,0]+matrix[1,0])
    print("Recall:",recall)
    specificity=matrix[1,1]/(matrix[0,1]+matrix[1,1])
    print("Specificity:",specificity)
    negative_pred_value=matrix[1,1]/(matrix[1,0]+matrix[1,1])
    print("Negative Predicted Value:",negative_pred_value)
    f1=2(precision*recall)/(precision+recall)
    print("F1 score:",f1)

    #the above code adds up the frequencies of the tps,tns,fps,fns and a matrix is formed
    return matrix